<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT22/blob/main/template-report-lab-X.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 4 : Approximation **
**Marc Hétier**

# **Abstract**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [1]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2020 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [2]:
# Load neccessary modules.
# from google.colab import files
import numpy as np
import scipy as sp
from copy import deepcopy

# **Introduction**

In this lab, we work on the approximation of function. The general background is an Hilbert space of function $V \subset \mathcal{L}^2(I)$, provided with the classical inner product :
$$(f, g) = \int_I f(x)g(x) dx $$

We also consider a subspace $V_N$ of $V$, of finite dimension $N$, and we search the projection of a function $f \in V$ on $V_N$. We note $P_Nf$ this projection.

Given a basis $\{\phi_i\}_{i = 1, \ldots ,N}$ of $V_N$, searching for $P_Nf$ is equivalent at searching its sets of coordinates $\{\alpha_j\}_{j = 1, \ldots ,N}$ such that 
$$ \forall i \;\;\; \sum_{j=1}^N \alpha_j (\phi_j, \phi_i) = (f, \phi_i) $$

This last equation is equivalent at the linear system
$$ A \alpha = b$$
where $\alpha = (\alpha_i)_i$, $A = ((\phi_j, \phi_i))_{i,j}$ and $b = (f, \phi_i)_i$.

Being able to find the projection of a function $f$ on a subspace is necessary to solve FEM problem. Then, the first part of our work will be dedicated to solving projection problem, and the second part will use this to solve a Poisson equation.

# **Method**

## Problem 1 : L2 projection to pw linear approwimation over a 1D mesh

In the following part, we assume that we are working with scalars functions. We also subdivise the interval $I$ into a mesh $(x_i)_{i=0}^{m+1}$ and we assume that it exists two families of basis. The first one consists of local basis for each sub-interval $[x_i, x_{i+1}]$, and the second one consists of a unique global basis for the entire interval $I$. More rigoursly, we define :
- for each $I_k$, a basis $(\lambda_{k,j})_{j =0}^{L_k}$
- the global basis function $(\phi_i)_i$, define using the local basis functions. This definition changes with the $(L_k)_k$ and if we want, or not a continuous approximation.

We create a function *assemble_system* which takes as input the function we want to approximate, the mesh and the local basis function. It returns the matrix $A$ and $b$ define above.
This function is very global, and use some sub functions which highly depends on the local basis we used, and if we want, or not, a continuous approximation. For those reasons, they are also pass as parameters.



In [3]:
def assemble_system(f, mesh, local_basis, get_local_to_global):
    """
    Input : a function f, a mesh under the form [x_0, ..., x_m+1] and
        list of local basis [[lambda_{k,1},..., lambda_{k,L_k}]...]
    Output : matrix A and b of the system
    """
    A = np.zeros()
    m = len(mesh) - 2
    for k in range(0,m+2):
        Lk = len(local_basis[k]) ## Since we are working in 1D, there is only two local basis function
        loc2glob = get_local_to_global(k)

        b_k = np.zeros(Lk)
        A_k = np.zeros((Lk,Lk))
        for i in range(0,Lk):
            b_k[i] = sp.integrate.quad(f, local_basis[k,i])
            for j in range(0,Lk):
                A_k[i,j] = sp.integrate.quad(local_basis[k,i], local_basis[k,j])
        
        A[loc2glob:Lk+1, loc2glob:Lk+1] = A_k
        b[loc2glob:Lk+1] = b_k



# **Discussion**